![](display_pics/title.png)

I buy a lot of clothes online, and I've noticed websites making an effort to show more diverse models. In recent months I've seen models of all sizes and races, when even a few years ago, the models all looked pretty much the same. However, this change isn't happening on all websites. Just from browsing, it's pretty clear that some websites are a lot more inclusive than others.

But... that's just anecdotal evidence. I'm more interested in a data-based answer. I love graphs-- how can I make a graph out of the skin tones of the models on a website? ([Click here to skip the process and see the graphs.](#graphs))

For a more manageable question I'll focus on the women's section of two websites that have different audiences: American Eagle, which sells affordable, casual clothes for teens and young adults, and Gucci, which is a high fashion company. The former has been actively touting their diversity with hashtags and email campaigns; the latter is not making nearly the same effort. Let's take a statistical look at how racially diverse these websites are.

![](display_pics/get_images.png)

The first step to getting models' skin tones is to get pictures of them. This is a simple as grabbing every image from the women's section of the website.

Web-scraping is a finnicky process that has to deal with a lot of site-specific JavaScript and CSS. I can write a generalizable function to get the image tags from each website, and another one to save the images to my machine, but to actually sift through the site's code, each website will have to be handled separately.

In [79]:
import requests
from bs4 import BeautifulSoup

In [80]:
HEADERS = {
    "User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'
}

In [87]:
def scrape_page(site, headers=HEADERS):
    page = requests.get(site, headers=headers)
    soup = BeautifulSoup(page.content, "lxml")
    img_tags = soup.find_all("img")
    return img_tags

def scrape_site(page_list, headers=HEADERS):
    all_tags = []
    for page in page_list:
        tags = scrape_page(page, headers)
        all_tags += tags
    return all_tags

def save_image(file_name, image_url):
    try:
        with open(file_name, "wb") as f:
            full_url = "http:{}?".format(image_url)
            response = requests.get(full_url)
            pic = response.content
            f.write(pic)
    except:
        print("Oops! The following URL failed to save:\n{}".format(image_url))

For American Eagle's website, I've only taken the pages that have images with the model's faces in them. For example, the "tailgate" section features mainly clothes hanging on a rack, and the "jeans" section has images that are taken from the waist down, so these pages have both been excluded when scraping.

In [82]:
AE_PAGES = [
    "https://www.ae.com/women-tops/web/s-cat/10049?cm=sUS-cUSD&navdetail=mega:womens:c1:p2"
    "https://www.ae.com/women-dresses/web/s-cat/1320034?cm=sUS-cUSD&navdetail=mega:womens:c1:p5",
    "https://www.ae.com/jumpsuits-rompers-women/web/s-cat/8490001?cm=sUS-cUSD&navdetail=mega:womens:c1:p6",
    "https://www.ae.com/matching-sets-women/web/s-cat/8420021?cm=sUS-cUSD&navdetail=mega:womens:c1:p7",
    "https://www.ae.com/women-don-t-ask-why/web/s-cat/6570005?cm=sUS-cUSD&navdetail=mega:womens:c1:p9"
    ]

In [83]:
ae_tags = scrape_site(AE_PAGES)
print("Recorded {} images from American Eagle's website".format(len(ae_tags)))

Recorded 5119 images from American Eagle's website


In [88]:
def process_ae_image(tag, count):
    file_name = "all_pics/scraped/ae/ae{0:04d}.jpg".format(count)
    try:
        full_url = tag["data-srcset"]
        url = full_url.split("?")[0]
        save_image(file_name, url)
        return True
    except:
        return False

count = 0
for tag in ae_tags:
    image_saved = process_ae_image(tag, count)
    if (image_saved):
        count += 1
        
print("Saved {} American Eagle images".format(count))

Saved 3273 American Eagle images


There are a lot of pictures that were recorded but not saved! These pictures are basically html garbage that is irrelevant to this project. A lot of them are logos, or swatches of color, and the `process_ae_image` function makes sure that they aren't saved. We're left with over 3,000 pictures of clothing and models from the American Eagle site.

The Gucci website is processed in a similar way, so that header and logo images aren't saved.

In [89]:
GUCCI_PAGES = [
    "https://www.gucci.com/us/en/ca/women/womens-ready-to-wear/womens-dresses-c-women-readytowear-dresses/1",
    "https://www.gucci.com/us/en/ca/women/womens-ready-to-wear/womens-coats-c-women-readytowear-coats",
    "https://www.gucci.com/us/en/ca/women/womens-ready-to-wear/womens-leather-casual-jackets-c-women-readytowear-leather-and-casual-jackets",
    "https://www.gucci.com/us/en/ca/women/womens-ready-to-wear/womens-jackets-c-women-readytowear-jackets",
    "https://www.gucci.com/us/en/ca/women/womens-ready-to-wear/womens-tops-shirts-c-women-readytowear-tops-and-shirts/1",
    "https://www.gucci.com/us/en/ca/women/womens-ready-to-wear/womens-sweaters-cardigans-c-women-readytowear-sweaters-and-cardigans",
    "https://www.gucci.com/us/en/ca/women/womens-ready-to-wear/womens-sweatshirts-t-shirts-c-women-readytowear-sweatshirts-and-tshirts/2",
    "https://www.gucci.com/us/en/ca/women/womens-ready-to-wear/womens-activewear-c-women-readytowear-activewear",
    "https://www.gucci.com/us/en/ca/women/womens-ready-to-wear/womens-skirts-c-women-readytowear-skirts/1",
    "https://www.gucci.com/us/en/ca/women/womens-ready-to-wear/womens-pants-shorts-c-women-readytowear-pants-and-shorts/1",
    "https://www.gucci.com/us/en/ca/women/womens-ready-to-wear/womens-denim-c-women-readytowear-denim"
    ]

In [90]:
gucci_tags = scrape_site(GUCCI_PAGES)
print("Recorded {} images from Gucci's website".format(len(gucci_tags)))

Recorded 1389 images from Gucci's website


In [96]:
def process_gucci_image(tag, count):
    file_name = "all_pics/scraped/gucci/gucci{0:04d}.jpg".format(count)
    try:
        url = tag["src"]
    except:
        try:
            url = tag["data-lazy-load-src"]
        except:
            return False
        
    if ("snapchat" in url):
        return False
    else:
        save_image(file_name, url)
        return True
    
count = 0
for tag in gucci_tags:
    image_saved = process_gucci_image(tag, count)
    if (image_saved):
        count += 1
        
print("Saved {} Gucci images".format(count))

Saved 1092 Gucci images


![](display_pics/get_faces.png)

I considered a lot of ways to extract the skin from these images. At first, I wanted to use a technique like [this](https://www.pyimagesearch.com/2014/08/18/skin-detection-step-step-example-using-python-opencv/) to get skin from any image-- even if it's just some ankles in a picture of jeans.

However, skin detection is tricky: it requires you to make assumptions about the person you are analyzing in order to determine upper and lower boundaries for colors that are similar to their skin tone. [Darker skin is not recorded well in pictures](https://www.youtube.com/watch?v=d16LNHIEJzs), and as a result, detecting skin for many people with a wide range of skin tones is tricky.

There's a workaround: rather than searching for skin, I can search for faces. [Facial recognition technology isn't perfect either](https://www.aclu.org/blog/privacy-technology/surveillance-technologies/amazons-face-recognition-falsely-matched-28), but its inconsistencies lie more in matching an image with an existing person, and less in pinpointing a face in an image. In other words, it'll be a better fit for this project. [`OpenCV`](https://docs.opencv.org/3.4.1/d7/d8b/tutorial_py_face_detection.html) has some great classifiers that I've made use of.

In [11]:
import numpy as np
import cv2
import os

In [108]:
def detect_faces(image, cascade_path="haarcascade_frontalface_default.xml"):
    face_cascade = cv2.CascadeClassifier(cascade_path)
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
            grayscale_image,
            scaleFactor=1.1,
            minNeighbors=5)
    return faces

def get_faces_in_image(image_path, company, image_number):
    image = cv2.imread(image_path)
    print(image_path)
    faces = detect_faces(image)
    face_number = 0
    for (x, y, width, height) in faces:
        cropped = image[y:y + height, x:x + width]
        file_name = "all_pics/cropped_faces/{0}/{1}{2:04d}_{3}.jpg".format(company, company, image_number, face_number)
        cv2.imwrite(file_name, cropped)
        face_number += 1
    
    # were there any faces found?
    return (face_number != 0)
    
def get_all_faces(company):
    directory = "all_pics/scraped/{}".format(company)
    image_number = 0
    for image in os.listdir(directory):
        image_path = "{}/{}".format(directory, image)
        found_faces = get_faces_in_image(image_path, company, image_number)
        if (found_faces):
            image_number += 1
    return image_number

First let's get the faces from the American Eagle models.

In [106]:
faces = get_all_faces("ae")
print("Found {} faces in American Eagle photos".format(faces))

Found 1181 faces in American Eagle photos


Now, let's get the Gucci faces.

In [111]:
faces = get_all_faces("gucci")
print("Found {} faces in Gucci photos".format(faces))

Found 369 faces in Gucci photos


Detecting faces is difficult, and the classification isn't always perfect. Scrolling through the folder of saved faces, it's easy to see that there are a few errors. I've flagged a lot of them by marking the number of faces found per picture. If two faces were detected in a single photo, there's been a mistake. So any image named `(ae|gucci)XXXX_1.jpg` should be checked manually. Below are a few of the images that were classified as faces.

![](display_pics/not_faces.png)

Maybe you're wondering: if I'm checking the photos manually for errors, what's the point of using automated facial classification anyway?

The time-consuming part of grabbing all of the faces from all of these models actually isn't facial recognition. Detecting faces is the kind of complicated task that humans are really good at: if someone flashed an image at me, I could tell you if it has a face a lot more quickly than most computer programs can. For me, the time consuming part would be opening an image, cropping it down to the right size, and saving the new image. That might take me as long as a few minutes per picture. However, cropping and saving images is the kind of tedious task that can be easily automated with a Python script. The task can be finished in a few minutes rather than several hours by coding almost everything, and double-checking the output.

After the Mistake Faces have all been deleted, I can run a quick script that will rename all the images so that they're in numerical order again. Another task where Python performs a lot more efficiently than humans!

In [121]:
def rename(company):
    directory = "all_pics/cropped_faces/{}".format(company)
    count = 0
    for image in os.listdir(directory):
        file_name = "all_pics/cropped_faces_renamed/{0}/{1}{2:04d}.jpg".format(company, company, count)
        image_path = "{}/{}".format(directory, image)
        image = cv2.imread(image_path)
        cv2.imwrite(file_name, image)
        count += 1

In [122]:
rename("ae")

In [117]:
rename("gucci")

Now that each image has been reduced to a face, it's time to identify the skin.

![](display_pics/get_skin.png)

An image is just a list of pixels (technically, a 2d array of pixels), and each pixel is a tiny square of solid color. There are a few ways to numerically represent color, but the one we'll use here is BGR: each pixel can be reduced to three integers that are the amount of blue, red, and green mixed to form that color. The integers range from 0 to 255. (Sometimes this is referred to as RGB.)

Since an image is just a collection of numbers, we can manipulate images with numerical operations. Consider taking the mean a photo that is predominantly skin (like a face!). Averaging together all of its pixels will give back one specific BGR value, which can approximate the skin tone of the model. 

In [125]:
def take_average(image):
    average_row_color = np.average(image, axis=0)
    average_color = np.average(average_row_color, axis=0)
    color_block = np.zeros((100, 100, 3), dtype=np.uint8)
    color_block[:, :] = average_color
    return color_block  

def get_average_colors(company):
    directory = "all_pics/cropped_faces_renamed/{}".format(company)
    for image in os.listdir(directory):
        image_path = "{}/{}".format(directory, image)
        face = cv2.imread(image_path)
        average_color = take_average(face)
        file_name = "all_pics/average_color/{}/{}".format(company, image)
        cv2.imwrite(file_name, average_color)

In [126]:
get_average_colors("ae")

In [127]:
get_average_colors("gucci")

![](display_pics/average_colors.png)

At first glance, this looks really cool! But... something seems off. A lot of the boxes are *really* dark. And others have a grayish tone that doesn't seem like a skin color. Let's take a look at some of the images up close.

![](display_pics/average_color.png)

The average colors actually don't approximate the skin tones of the models at all. There are too many other elements getting in the way. In the images above, the models' hair and accessories pull the average away from the skin tone. To more concisely state the problem: there are too many non-skin pixels. 

Averaging would work really well if we could start by grabbing only the skin pixels from the image, and then working with those. Based on the features of a pixel (its blue, green, and red values) we want to know whether it is a skin pixel, or non-skin. Just like that, this has become an unsupervised classification problem. It's *classification* because there is a discrete set of values the outcome can take (two, in this case). It's *unsupervised* because we have no correctly-labelled training data to use. This type of problem is great to solve with k-means, which identifies points that have similar features, and groups them into k clusters. In this case, we are looking for two clusters: one is skin, and the other is everything else.

Let's do some preliminary clustering. For each face, I've fit a k-means classifier, where k=2. Then I paint each pixel according to how it's been classified: any pixel assigned a 0 is colored red, and any pixel assigned 1 is colored blue. (The code for creating and saving these images is tedious at best, so I've left it out of this notebook.)

![](display_pics/red_and_blue.png)

The clusters map to the skin pretty well, but we can use a pretty simple trick to make the classification more accurate. Very few pixels on a person will be pure white, whereas the pixels that make up the background are all very close to white. Here, "close" refers to a pixel being numerically close to the value that represents white: (255, 255, 255). Before fitting the classifier, we can remove all of the white pixels. The subset of less extreme values will make the new means that are learned by the classifier more accurate.

Below I've fit a k-means classifier (k=2) on the non-white pixels in an image. In the images, the white pixels that were left out when fitting the classifier are colored white, the pixels tagged "0" are colored red, and the pixels tagged "1" are colored blue.

![](display_pics/red_and_blue_white_background.png)

So now we have three groups of pixels: background, skin, and other. Really, these are just two groups: skin, and non-skin. We need to combine the background pixels and other pixels into a single group. However, that is easier said than done. From the pictures above, you can see that sometimes the skin pixels are tagged "0" (colored red) and sometimes they are tagged "1" (colored blue). Right now, our classifier has no way of assigning the meaningful labels of skin and non-skin to these arbitrary tags.

To programmatically decide which of the tags is associated with skin, we can make an assumption about the images: a square of pixels directly in the center of the image will probably be mainly skin. So we can grab a central patch of the photo, and classify all of it's pixels. The dominant label (either 0 or 1) is the "skin label," and the picture can be colored according to that.

Now we can squash the three original categories into just two categories, and look at the results. I've colored the skin pixels blue, and the non-skin pixels white.

![](display_pics/blue_skin.png)

Once the skin pixels have been identified, we can use those to calculate the average skin tone. Below are the averages calculated using just skin, compared to the original averages that were calculated using every pixel in the image. The new average is a much better approximation.

![](display_pics/average_skin.png)

Here's the progression of an image from the face being detected, to the final average skin tone, with the original average for comparison.

![](display_pics/start_to_end.png)

Below is the code that completes all of these steps for one face in order to calculate its average skin color. (I've omitted the lines that create colored images like above and save them into directories.)

```python
from sklearn.cluster import KMeans
from collections import Counter

def cluster_face(image, white_threshold):
    
    # save a patch of pixels from the center of the pic to identify the skin label
    (height, width, three) = image.shape
    (center_x, center_y) = (width // 2, height // 2)
    patch = image[center_x - 10 : center_x + 10, center_y - 10 : center_y + 10]
    flattened_patch = patch.transpose(2, 0, 1). reshape(3, -1).transpose()
    
    # remove white pixels, and fit a classifier
    flattened = image.transpose(2, 0, 1).reshape(3, -1).transpose()
    filtered = np.array([pixel for pixel in flattened if pixel.sum() < white_threshold])
    k_means = KMeans(n_clusters=2)
    k_means.fit(filtered)
    
    # identify the skin label
    patch_labels = k_means.predict(flattened_patch)
    skin_label = Counter(patch_labels).most_common()[0][0]
    
    # save the average skin pixel
    all_labels = k_means.predict(filtered)
    skin_mask = (all_labels == skin_label)
    skin_pixels = filtered[skin_mask]
    average_color = np.average(skin_pixels, axis=0)
    return average_color
```

![](display_pics/get_light.png)

Now there's a single BGR value for each model representing her skin tone. But it's tricky to visualize triples. What's more, BGR values are for machines, and not for humans. When I look at how diverse a website seems, I'm not thinking about the levels of blue, green, and red in her skin tone, I'm thinking about how light or dark her skin is. So before trying to graph these machine-readable values, we need to translate them to numbers that make sense to humans.

Inspired by [this article](https://pudding.cool/2018/06/makeup-shades/), each BGR triple can be converted to a [HLS triple](https://www.w3schools.com/colors/colors_hsl.asp). This means that instead of recording the blue, green, and red levels of a color, we record it's hue, lightness, and saturation. From there, we have a single numerical value (that is, a scalar rather than a vector) that represents how light the model's skin is.

In [5]:
import pandas as pd
import colorsys

In [113]:
color_values = pd.read_csv("bgr.csv")
color_values.head()

,image_name,company,b,g,r
0,ae0000.jpg,ae,106,130,188
1,ae0001.jpg,ae,113,133,188
2,ae0002.jpg,ae,126,158,217
3,ae0003.jpg,ae,151,172,210
4,ae0004.jpg,ae,152,170,211


In [114]:
h_col = []
l_col = []
s_col = []

for index, row in color_values.iterrows():
    r = row["r"] / 255
    g = row["g"] / 255
    b = row["b"] / 255
    (h, l, s) = colorsys.rgb_to_hls(r, g, b)
    h_col.append(h)
    l_col.append(l)
    s_col.append(s)
    
color_values["h"] = h_col
color_values["l"] = l_col
color_values["s"] = s_col

color_values = color_values.sort_values(by=["l"])
color_values.head()

,image_name,company,b,g,r,h,l,s
690,ae0691.jpg,ae,25,41,84,0.045198,0.213725,0.541284
612,ae0613.jpg,ae,26,41,87,0.040984,0.221569,0.539823
1120,gucci0086.jpg,gucci,40,55,87,0.053191,0.249020,0.370079
692,ae0693.jpg,ae,44,57,95,0.042484,0.272549,0.366906
333,ae0333.jpg,ae,43,59,102,0.045198,0.284314,0.406897


Now the data is ready to plot.

![](display_pics/visualizing_diversity.png)

(*The code for making graphics is very precise and specific, and not very instructive, so I've omitted it to focus on the graphics themselves.*)

First let's look at a palette of all the shades recorded, to see if there are any interesting patterns. The shades from American Eagle are pictured first, and Gucci next.

![](display_pics/ae_all.jpg)

![](display_pics/gucci_all.jpg)

There's a pretty clear difference between the two companies: there are more dark shades from the Gucci website (especially percentage wise!) but you can see almost exactly where the dark shades drop off, and paler shades take over. In the American Eagle palette, there are plenty of "medium" colors. If we plot the shades, the drop-off in Gucci shades is very clear. American Eagle has a smoother curve.

(The color for each bar is the average BGR value for all of the colors in that fall into that bucket of lightness values, so the color of any bar on the AE graph will be slightly different from the color of the corresponding bar on the Gucci graph.)

<a id="graphs"></a>
![](display_pics/hists.png)

Finally, we can take a closer look at the shades in each of the buckets of lightness values by listing them all out, rather than averaging them together. Even though the format of the graphic is different, Gucci's bimodal distribution is still apparent.

![](display_pics/new_chart.png)

![](display_pics/takeaways.png)

I was really surprised to see the overall shapes of the distributions. I did not expect a bimodal distribution at all, but after taking a look at the style prevalent on Gucci's website, it makes sense. The models they use have strikingly dramatic features, from their cheekbones to the color of their skin. This means selecting very dark and very light models. American Eagle's site, on the other hand, depicts models that fall into all sorts of ranges, which is why the distribution is smoother.

This analysis also gave me a chance to look at how I assess a website's diversity. Just from scrolling, I decided that American Eagle's site contained a diverse range of models, and Gucci's was much less diverse. This is because while scrolling through the American Eagle site, I saw many more women of color, without thinking about the ratio of darker skinned models to total models. On Gucci's site, I saw fewer women of color, but it turns out that they actually make up a large percentage of the total images.

![](display_pics/flaws.png)

This is not a perfect analysis. However, each time I encountered a problem, I tried to address in the most appropriate way. Below are some of the things that I considered while working on this project, and some things that I would change or add if I had the chance to expand this project.


**I didn't touch "featured" photos**

The biggest thing that I would change about this project if I work on it in the future is to look at which models are "featured" at the top of each page. These models typically have a whole photo shoot, and their images are placed in prominent locations. By focusing only on the standard pictures of clothing, I brushed over another area where I could have looked at diversity.

**Using multiple pictures of one model is different from using multiple pictures of multiple models**

Many of the photos that I've used are repeats: they are different photos taken of the same model. At first, I considered only analyzing the distinct faces. Let's say a company has fifty white models and one black model. They post one picture each of the white models, and twenty of the black model. The code in this notebook would show that the website is pretty diverse, and it is, in one sense of the definition. But I wouldn't be comfortable calling a company that hires fifty times more white models than black models "diverse"-- it's a little more complicated than that. At the same time, the reverse could happen. A company could hire ten white models, and ten black models, and then post five hundred pictures featuring the white models, and ten pictures featuring the black models. In one type of analysis, this website would shine. In another type of analysis, they wouldn't. In the end, I decided that what is important to me is how often women of color are featured on the site, so that's what I focused on. But with more time, I would love to work with facial classification and identify the roster of models that a website has.

**This analysis only includes two websites**

Originally, I wanted to use close to a dozen different websites (it seems like there are infinite clothing websites nowadays), but since it was very difficult to generalize the web-scraping functions, the bulk of this project would have been spent wrangling whatever JavaScript the web designers implemented. I wanted to spend more time on the machine learning techniques and visualization.

**The websites have a very different number of images**

The nature of the companies (American Eagle sells a huge selection of casual clothes, while Gucci sells a curated collection of high fashion garments) means that the sites have different numbers of pictures. Some would say this is comparing apples to oranges. I'd argue that representation in fashion is important on all levels, and by using density rather than count in my histograms, I handle the difference well. What's more, the Gucci website still has a huge number of images: 300+ is a reasonable sample size for this type of analysis.

**Only racial diversity is addressed**

In fashion, ethnicity is only one area where diversity is lacking. Many companies favor thin models over plus-sized women, even when plus-sized models more accurately reflect the average customer. Whenever there are multiple areas of under-representation, there is also the issue of intersectionality. Perhaps a website has a reasonable amount of models of color and plus-sized models, but no plus-sized models of color. These are all important issues, but size is trickier to detect from images. I considered using edge detection to outline the women's bodies and compare this to the image size, and I considered scraping the model's names and sizes from different sources, but in the end decided that focusing on one area of underrepresentation was a more manageable task for a project this size.

**Only a subset of racial diversity is addressed**

While I look at the skin tones of the models, I don't look at their explicit race. A light-skinned Asian model may appear the same color as a white person; a mixed race model may appear the same color as an Indian person. There were two reasons that I was okay with looking at color rather than demographics. First, detecting race from a photo is messy and complicated. Second, skin color is an issue even within certain races. You can read about color-related beauty standards for [Asian](https://en.wikipedia.org/wiki/Light_skin_in_Japanese_culture), [Indian](https://thewire.in/culture/69275-tannishtha-chatterjee-comedy-night-bachao), and [Black](http://theconversation.com/black-americas-bleaching-syndrome-82200) women.

**I only looked at the women's section of each site**

Since the motivation for this project was anecdotal, and I don't ever look at the men's section, I didn't have the same motivation to analyze the diversity in men's photos. What's more, the men's sections of these sites had significantly fewer products, and I wanted to work with a lot of data.

![](display_pics/refs.png)

I started this project with a lot of theoretical knowledge, but very little practical knowledge. This meant a lot of googling. While I can't post every StackOverflow link that helped me solve an error, the following pages were instrumental in translating my high level understanding into workable code.

* [Implementing facial recognition with `OpenCV`](https://realpython.com/face-recognition-with-python/)
* [Scraping images from a website with `BeautifulSoup`](https://stackoverflow.com/questions/18304532/extracting-image-src-based-on-attribute-with-beautifulsoup)
* [Using thresholds to pinpoint white pixels](https://www.mathworks.com/matlabcentral/answers/328835-how-can-i-extract-the-white-pixels-of-an-image)

# Contact me!

Below are lots of ways to talk to me about this or other projects!

* handamalaika (at) gmail.com
* [Twitter](twitter.com/malicodes)
* [LinkedIn](https://www.linkedin.com/in/malaika-handa/)
* [Resume](https://drive.google.com/file/d/1ammnktVHH-GdZ2UOsGCsUS_VM3w8csh8/view?usp=sharing)